# Matchmaker

The Matchmaker was an honored profession in past cultures, serving the valuable purpose of pairing off men and women in hopes of a long and successful relationship.  The Matchmaker would carefully consider the characteristics of each partner to determine which pairs would be compatible.

Figure 1 shows six men and six women each with varying hair color and eye color.  A man or woman will only accept a partner that has at least one of these traits in common.  For example, Man A and Woman 5 would make a matching pair because they have at least one trait in common (same hair color).  However Man A and Woman 1 would not make a matching pair because they do not have any traits in common.

Question:  What pairings of men and women allow for everyone to have a partner with at least one trait in common?  (There are several correct answers.)

### References

 * The PuzzlOr - Matchmaker (http://puzzlor.com/2011-06_Matchmaker.html) .

<center>
<img src="http://puzzlor.com/images/2011-06%20Matchmaker_v3.gif">   
</center>

We begin with importing the library we going to use

In [ ]:
!pip install docplex==0.0.24

In [ ]:
from docplex.mp.model import Model

Let's define now our man's and women's characteristics

In [ ]:
color_hair = ['green','red','black'] # all the color of hair
color_eyes= ['green','bleu','red']   # all the color of eyes

man = {'A':[2,1],'B':[1,2],'C':[1,3],'D':[2,1],'E':[3,1],'F':[1,2]}
female = {'1':[1,3],'2':[2,1],'3':[3,2],'4':[3,3],'5':[2,3],'6':[3,3]}

Let's define now some useful variable for our model

In [ ]:
#matrix of all the combinaison between a man and women
Comb = [['A1','A2','A3','A4','A5','A6'],
        ['B1','B2','B3','B4','B5','B6'],
        ['C1','C2','C3','C4','C5','C6'],
        ['D1','D2','D3','D4','D5','D6'],
        ['E1','E2','E3','E4','E5','E6'],
        ['F1','F2','F3','F4','F5','F6']]

#transpose the matrix below
tComb = [[Comb[j][i] for j in range(0, 6)] for i in range(0, 6)]

#Matrix comb in other form
Comb2 = ['A1','A2','A3','A4','A5','A6',
        'B1','B2','B3','B4','B5','B6',
        'C1','C2','C3','C4','C5','C6',
        'D1','D2','D3','D4','D5','D6',
        'E1','E2','E3','E4','E5','E6',
        'F1','F2','F3','F4','F5','F6']

#Function test_match that test if a man and women match toghether
def test_match(a):
    m = a[0]
    f = a[1]
    return int(man[m][0]==female[f][0]) + int( (man[m][1]==female[f][1]))

Now let's define our model and solve it

In [ ]:
from docplex.mp.environment import Environment
env = Environment()
env.print_information()

In [ ]:
from docplex.mp.context import DOcloudContext
context = DOcloudContext(url="PUT_YOUR_DOCLOUD_URL_HERE", 
                         api_key="PUT_YOUR_DOCLOUD_KEY_HERE")

In [ ]:
mdl = Model(docloud_context=context)

Comb_vars = mdl.binary_var_dict(Comb2, name="is_match")


#contraint : every man have one women
for i in Comb:
    mdl.add_constraint(mdl.sum(Comb_vars[h] for h in i) <= 1)

#contraint : every women to have one man
for i in tComb:
    mdl.add_constraint(mdl.sum(Comb_vars[h] for h in i) <= 1)
    
#contraint : a man have a women if they match
for i in Comb:
    mdl.add_constraint(mdl.sum(Comb_vars[h] * test_match(h) for h in i) >= 1)
    
#for h in Comb2:
#    ind1 = mdl.add_indicator(Comb_vars[h],  Comb_vars[h] * test_match(h) >= 1 , active_value=1 )

        
# our objective is to find the maximum couples
mdl.maximize(mdl.sum(Comb_vars[h]* test_match(h) for h in Comb2))

#mdl.print_information()
#mdl.prettyprint()

ok = mdl.solve()

mdl.print_solution()

Let's add a nice print to our result

In [ ]:
if not ok:
    print("Matchmaker model fails")
else:
        total_points = mdl.objective_value
        couples = [h for h in Comb2 if float(Comb_vars[h]) ==1]
        print("we need to match thoses couples:")
        for i in couples:
            print("the man {} with the women {}" .format(i[0],i[1]))